## Calculate HU Accuracy

In [69]:
import os
import numpy as np
import cv2

def calculate_hu_accuracy(ct_folder, cbct_folder):
    """
    Calculate Mean Absolute Error (MAE) and Root Mean Square Error (RMSE)
    across pairs of images in two folders with matching filenames (PNG files).

    Args:
        ct_folder (str): Path to the folder containing CT images.
        cbct_folder (str): Path to the folder containing CBCT images.

    Returns:
        dict: Dictionary containing 'mean_mae', 'std_mae', 'mean_rmse', and 'std_rmse'.
    """
    # Get the list of image filenames (assuming same set in each folder)
    ct_images = sorted([
        f for f in os.listdir(ct_folder)
        if f.lower().endswith('.png')
    ])
    
    cbct_images = sorted([
        f for f in os.listdir(cbct_folder)
        if f.lower().endswith('.png')
    ])
    
    mae_values = []
    rmse_values = []
    
    for ct_img_name, cbct_img_name in zip(ct_images, cbct_images):
        # Build full file paths
        ct_path = os.path.join(ct_folder, ct_img_name)
        cbct_path = os.path.join(cbct_folder, cbct_img_name)
        
        # Read images as float32 (unchanged ensures exact reading type, e.g. 16-bit if present)
        ct_img = cv2.imread(ct_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
        cbct_img = cv2.imread(cbct_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
        
        # Compute the absolute difference and squared difference
        diff = ct_img - cbct_img
        abs_diff = np.abs(diff)
        squared_diff = diff ** 2
        
        # Calculate MAE for this pair
        mae = np.mean(abs_diff)
        mae_values.append(mae)
        
        # Calculate RMSE for this pair
        rmse = np.sqrt(np.mean(squared_diff))
        rmse_values.append(rmse)
    
    # Compute the overall mean and standard deviation of MAEs and RMSEs
    mean_mae = np.mean(mae_values)
    std_mae = np.std(mae_values)
    mean_rmse = np.mean(rmse_values)
    std_rmse = np.std(rmse_values)
    
    return {
        'mean_mae': mean_mae,
        'std_mae': std_mae,
        'mean_rmse': mean_rmse,
        'std_rmse': std_rmse
    }

if __name__ == "__main__":
    ct_folder_path = "../data/crop/CT"
    cbct_folder_path = "../data/crop/CBCT"
    
    results = calculate_hu_accuracy(ct_folder_path, cbct_folder_path)
    
    # Print out the results
    print(f"MAE = {results['mean_mae']:.2f} ± {results['std_mae']:.2f}")
    print(f"RMSE = {results['mean_rmse']:.2f} ± {results['std_rmse']:.2f}")

MAE = 1.32 ± 0.70
RMSE = 5.62 ± 1.78


## Calculate image quality metrics

In [67]:
import os
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim

def calculate_image_quality_metrics(ct_folder, cbct_folder, win_size=7):
    """
    Calculate PSNR and SSIM across pairs of matching images in two folders.
    Allows specifying the SSIM window size (win_size).

    Args:
        ct_folder (str): Path to the folder containing CT images.
        cbct_folder (str): Path to the folder containing CBCT images.
        win_size (int, optional): The side-length of the sliding window used 
                                  in SSIM calculation. Must be odd. Defaults to 7.

    Returns:
        dict: Dictionary containing the mean and standard deviation of 
              PSNR and SSIM values across all paired images.
    """
    # Gather PNG filenames (adjust if you have other extensions)
    ct_images = sorted([
        f for f in os.listdir(ct_folder)
        if f.lower().endswith('.png')
    ])
    cbct_images = sorted([
        f for f in os.listdir(cbct_folder)
        if f.lower().endswith('.png')
    ])

    psnr_values = []
    ssim_values = []

    for ct_img_name, cbct_img_name in zip(ct_images, cbct_images):
        ct_path = os.path.join(ct_folder, ct_img_name)
        cbct_path = os.path.join(cbct_folder, cbct_img_name)

        # Read images (unchanged to preserve bit depth), then cast to float32
        ct_img = cv2.imread(ct_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
        cbct_img = cv2.imread(cbct_path, cv2.IMREAD_UNCHANGED).astype(np.float32)

        # 1) Calculate PSNR
        #    PSNR = 20 * log10(MAX) - 10 * log10(MSE)
        #    We'll assume an 8-bit max (255.0). 
        mse = np.mean((ct_img - cbct_img) ** 2)
        if mse == 0:
            # Images are identical
            psnr_value = float('inf')
        else:
            psnr_value = 20 * np.log10(255.0 / np.sqrt(mse))
        psnr_values.append(psnr_value)

        # 2) Calculate SSIM
        #    If images are smaller than win_size, pass a smaller window or set dynamically
        #    data_range should match the actual intensity range
        data_range = (max(ct_img.max(), cbct_img.max()) 
                      - min(ct_img.min(), cbct_img.min()))

        if data_range == 0:
            # Images are identical
            ssim_value = 1.0
        else:
            # Make sure win_size is valid for the current image dimensions.
            # We choose the minimum of (win_size, min_image_dim), ensuring an odd size >= 3
            min_dim = min(ct_img.shape)
            actual_win_size = min(win_size, min_dim)
            # Ensure it remains an odd number:
            if actual_win_size % 2 == 0:
                actual_win_size -= 1
            # The smallest recommended window size is 3x3
            actual_win_size = max(actual_win_size, 3)

            ssim_value = ssim(ct_img, cbct_img, data_range=data_range, win_size=actual_win_size)
        ssim_values.append(ssim_value)

    # Calculate means and standard deviations
    mean_psnr = np.mean(psnr_values)
    std_psnr = np.std(psnr_values)
    mean_ssim = np.mean(ssim_values)
    std_ssim = np.std(ssim_values)

    return {
        'mean_psnr': mean_psnr,
        'std_psnr': std_psnr,
        'mean_ssim': mean_ssim,
        'std_ssim': std_ssim
    }

if __name__ == "__main__":
    ct_folder_path = "../data/crop/CT"
    cbct_folder_path = "../data/crop/CBCT"
    
    # If images are smaller than 7x7, try a smaller win_size
    metrics = calculate_image_quality_metrics(ct_folder_path, cbct_folder_path, win_size=7)
    
    # Print a simple summary of results
    print(f"PSNR: {metrics['mean_psnr']:.2f} ± {metrics['std_psnr']:.2f}")
    print(f"SSIM: {metrics['mean_ssim']:.4f} ± {metrics['std_ssim']:.4f}")

PSNR: 33.41 ± 2.03
SSIM: 0.9853 ± 0.0032
